# Imports

In [1]:
import keras
import numpy as np
import pandas as pd
import transformers
import tensorflow as tf
import tqdm.notebook as tqdm
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'keras'

In [1]:
#import keras
import os
import datasets
import numpy as np
import pandas as pd
import transformers
import sklearn.metrics
#import tensorflow as tf
import tqdm.notebook as tqdm
import sklearn.model_selection
import matplotlib.pyplot as plt

#pytorch

import torch
from torcheval.metrics import MultilabelAUPRC
from torchmetrics.classification import MultilabelAUROC
#from torchsummary import summary

from sklearn.model_selection import train_test_split

/home/bob1ch/Рабочий стол/NN-NLP/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_PATH = 'models'
MAX_TOKENS = 32

# Dataset

Get the dataset from [here](https://tatoeba.org/en/downloads). Preferably use russian to english translations.

Use a custom tokenizer that can add bos and eos tokens (pass `add_special_tokens=True` when calling the tokenizer to add them).

In [3]:
class Tokenizer(transformers.GPT2Tokenizer):

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        if token_ids_1 is None:
            return [self.bos_token_id, *token_ids_0, self.eos_token_id]

        return [self.bos_token_id, *token_ids_0, self.bos_token_id, *token_ids_1, self.eos_token_id]

In [4]:
tokenizer = Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token_id = tokenizer.eos_token_id

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'Tokenizer'.
/home/bob1ch/Рабочий стол/NN-NLP/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
def to_tokenize(data):
    return tokenizer(data, max_length=MAX_TOKENS, truncation=True, padding=True)['input_ids']

Since the dataset is rather large, you can omit the validation dataset and just use a set of test sentences after the training.

Create a dataset that returns the following
* A pair of tensors `((None, L), (None, P))` -- input sequence of tokens and output sequence of tokens to be fed into decoder (this should start with the BOS token)
* A tensor `(None, P)` -- output sequence of tokens to be predicted (this should end with EOS token)
* A tensor `(None, P)` -- a masking tensor marking padded tokens with 0

In [6]:
data = pd.read_csv('seq2seq_dataset.tsv', 
            sep='\t', 
            on_bad_lines='skip',
            names=['id_1', 'rus', 'id_2', 'eng'])[['rus', 'eng']]#тут скип, потомучта какое-то говно возникало

X, y = to_tokenize(data['rus'].to_list()), to_tokenize(data['eng'].to_list())

In [7]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.7, shuffle=True)

In [8]:
def get_dataloader(batch, X, y):
    inp = torch.LongTensor(X).to('cuda')
    out_BOS = torch.LongTensor(y).to('cuda')[:, :-1]
    out_EOS = torch.LongTensor(y).to('cuda')[:, 1:]
    masking_PAD = (out_EOS != tokenizer.pad_token_id).to(torch.int32).to('cuda')
    masking_PAD[:, 0] =  1 #BOS==EOS поэтому тут ручками доделал
    data = torch.utils.data.TensorDataset(inp,
                                          out_BOS,
                                          out_EOS,
                                          masking_PAD)
    return torch.utils.data.DataLoader(data, batch_size=batch, shuffle=True)

In [9]:
data = get_dataloader(64, X_train, y_train)

# Model

Create a model for training. The model should have two inputs: input sequence `(None, L)` and output sequence`(None, P)`. The model output is a single tensor `(None, P)` logits (or probabilities) of the next token predicted for each input one.

In [10]:
class Encoder(torch.nn.Module):

    def __init__(self, units: int, n_tokens: int, n_stacks: int, bidirectional: bool, name: str):
        super(Encoder, self).__init__()
        self.name = name
        self.units = units
        self.bidirectional = bidirectional
        
        self.embeddings = torch.nn.Embedding(n_tokens, units)
        self.rnn = torch.nn.GRU(units, units, num_layers=n_stacks, bidirectional=bidirectional, batch_first=True) #можно поменять

    def forward(self, text):
        embeds = self.embeddings(text)      
        out, h = self.rnn(embeds)

        return out, h

#https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#the-decoder
class Decoder(torch.nn.Module):

    def __init__(self, units: int, n_tokens: int, n_stacks: int, name: str): 
        super(Decoder, self).__init__()
        self.name = name
        self.units = units
        #self.bidirectional = bidirectional
        
        self.embeddings = torch.nn.Embedding(n_tokens, units)
        self.rnn = torch.nn.GRU(units, units, num_layers=n_stacks, batch_first=True)
        self.FC = torch.nn.Linear(units, n_tokens)

    def forward(self, enc_outs, enc_h, target=None):
        #enc_outs нужен чтобы батч узнать
        batch_size = enc_outs.size(0)
        dec_inp = torch.empty(batch_size, 1, dtype=torch.long, device='cuda').fill_(tokenizer.bos_token_id)
        dec_h, dec_out = enc_h, []
        
        #цикл генерации токенов, подумой можно ли остановить, если пришел паддинг
        for i in range(1, MAX_TOKENS):
            out, dec_h = self.apply_rnn(dec_inp, dec_h)
            dec_out.append(out)
            dec_inp = target[:, i] if target else out.argmax(dim=-1).detach()
            
            
        dec_out = torch.cat(dec_out, dim=1)
        #dec_out = torch.nn.functional.log_softmax(dec_out, dim=-1)
        #dec_out = torch.nn.functional.log_softmax(dec_out, dim = -1)
        return dec_out

    def apply_rnn(self, inp, h):
        embeds = self.embeddings(inp)      
        out, h = self.rnn(embeds)
        out = self.FC(out)

        return out, h

class Seq2Seq(torch.nn.Module):

    def __init__(self, units: int, n_tokens: int, n_stacks: int, bidirectional: bool, name: str):
        super(Seq2Seq, self).__init__()
        self.encoder = Encoder(units, n_tokens, n_stacks, bidirectional, name+'_encoder').to('cuda')
        self.decoder = Decoder(units, n_tokens, n_stacks, name+'_decoder').to('cuda')

    def forward(self, text, target=None):
        enc_outs, enc_h = self.encoder(text)
        dec_out = self.decoder(enc_outs, enc_h, target)
        
        return dec_out

In [11]:
encoder = Encoder(32, tokenizer.vocab_size, 1, False, 'encoder').to('cuda')
decoder = Decoder(32, tokenizer.vocab_size, 1, 'decoder').to('cuda')
text = next(iter(data))[0][:4]
enc_outs, enc_h = encoder(text)

In [12]:
dec_out = decoder(enc_outs, enc_h)

In [13]:
dec_out

tensor([[[ 0.0764, -0.1818, -0.0432,  ...,  0.0474,  0.1417, -0.1392],
         [-0.0058, -0.0145, -0.0555,  ...,  0.2050, -0.0307, -0.1863],
         [ 0.0986, -0.0988, -0.0547,  ...,  0.1074, -0.1742, -0.2166],
         ...,
         [-0.2084,  0.1315,  0.0560,  ..., -0.1206,  0.1176, -0.0019],
         [-0.2331,  0.0623,  0.0282,  ...,  0.0081,  0.0164, -0.1098],
         [-0.0543,  0.1132, -0.0252,  ..., -0.0314, -0.0231, -0.2556]],

        [[ 0.0764, -0.1818, -0.0432,  ...,  0.0474,  0.1417, -0.1392],
         [-0.0058, -0.0145, -0.0555,  ...,  0.2050, -0.0307, -0.1863],
         [ 0.0986, -0.0988, -0.0547,  ...,  0.1074, -0.1742, -0.2166],
         ...,
         [-0.2084,  0.1315,  0.0560,  ..., -0.1206,  0.1176, -0.0019],
         [-0.2331,  0.0623,  0.0282,  ...,  0.0081,  0.0164, -0.1098],
         [-0.0543,  0.1132, -0.0252,  ..., -0.0314, -0.0231, -0.2556]],

        [[ 0.0764, -0.1818, -0.0432,  ...,  0.0474,  0.1417, -0.1392],
         [-0.0058, -0.0145, -0.0555,  ...,  0

In [14]:
model = Seq2Seq(32, tokenizer.vocab_size, 1, False, 'encoder')
model(text)

tensor([[[-0.2635,  0.0122,  0.1862,  ...,  0.1534, -0.1088,  0.0057],
         [ 0.0089,  0.1840,  0.1620,  ...,  0.2109, -0.1404, -0.1136],
         [ 0.2055,  0.3816,  0.1435,  ...,  0.0129, -0.0054, -0.1813],
         ...,
         [-0.1313,  0.0548,  0.2401,  ...,  0.2257, -0.1576,  0.1436],
         [-0.3108,  0.2134,  0.0107,  ...,  0.1620, -0.1226,  0.1444],
         [-0.2219,  0.4159,  0.0727,  ...,  0.2682,  0.0033,  0.3220]],

        [[-0.2635,  0.0122,  0.1862,  ...,  0.1534, -0.1088,  0.0057],
         [ 0.0089,  0.1840,  0.1620,  ...,  0.2109, -0.1404, -0.1136],
         [ 0.2055,  0.3816,  0.1435,  ...,  0.0129, -0.0054, -0.1813],
         ...,
         [-0.1313,  0.0548,  0.2401,  ...,  0.2257, -0.1576,  0.1436],
         [-0.3108,  0.2134,  0.0107,  ...,  0.1620, -0.1226,  0.1444],
         [-0.2219,  0.4159,  0.0727,  ...,  0.2682,  0.0033,  0.3220]],

        [[-0.2635,  0.0122,  0.1862,  ...,  0.1534, -0.1088,  0.0057],
         [ 0.0089,  0.1840,  0.1620,  ...,  0

In [ ]:
def get_model(
    units: int,
    n_tokens: int,
    n_labels: int,
    n_stacks: int = 1,
    bidirectional: bool = False,
    name: str | None = None,
    cell_type: type[keras.layers.Layer] = keras.layers.LSTMCell
) -> keras.Model:
    '''Creates a model with RNN architecture for sequence to sequence classification.

    Arguments:
        units: dimensionality of RNN cells
        n_tokens: number of tokens in the tokenizer dictionary
        n_labels: number of labels to be predicted
        n_stacks: number of RNN cells in the stack (1 -- no stacking)
        bidirectional: whether or not the model is bidirectional
        name: the model name
        cell_type: type of a cell to use, either keras.layers.LSTMCell or keras.layers.GRUCell

    Returns:
        The model'''
    ...

Try to add attention to your model (for example [additive attention](https://keras.io/api/layers/attention_layers/additive_attention/)), does it perform better?

# Training

Train your model using teacher forcing. The idea is that the model predicts the next token that should follow, so one part of the model (called encoder) reads the text and output some state containing information about the text read. The other part of the model (called decoder) reads and already generated text (or in case of the teacher forcing the expected output) and predicts the next token for each one. 

In [201]:
False and (1 + 1)

False

In [21]:
def train(model, optimizer, loss_f, dataloader, target = None):

    model.train()
    running_loss = 0

    for i, batch in enumerate(dataloader, start=1):
        inp, out_BOS, out_EOS, masking_PAD = batch
        seq_pred = model(inp) #out_EOS
        loss = loss_f(seq_pred.view(-1, seq_pred.size(-1)), 
                      out_EOS.view(-1))
        running_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if i % (len(dataloader) // 4) == 0:
            print(f'batch{i} cur_loss {running_loss / i}')
        
    return running_loss / len(dataloader)

In [22]:
EPOCHS = 4
model = Seq2Seq(32, tokenizer.vocab_size, 1, False, 'seq2seq')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_f = torch.nn.CrossEntropyLoss()
for epoch in range(EPOCHS):
    loss = train(model, optimizer, loss_f, data)
    print(loss)

batch859 cur_loss 3.2021246064666817
batch1718 cur_loss 2.6535813154802335
batch2577 cur_loss 2.4703910340622595
batch3436 cur_loss 2.3790593242339955
2.3787346945560475
batch859 cur_loss 2.10327064088948
batch1718 cur_loss 2.1000013313970687
batch2577 cur_loss 2.10033908998684
batch3436 cur_loss 2.100938254865418
2.100955730401073
batch859 cur_loss 2.095481052665133
batch1718 cur_loss 2.0988480241251493
batch2577 cur_loss 2.098907358941525
batch3436 cur_loss 2.1000755715148136
2.100088653608823
batch859 cur_loss 2.0998508286281847
batch1718 cur_loss 2.099096388700261
batch2577 cur_loss 2.0987796247306147
batch3436 cur_loss 2.0996129776462826
2.0995605701137054


# Testing

Make a function for text translation. Translate some text and evaluate model performance.

Take note that your model is set for training. During the inference process you will have to use parts of the model independently (including the RNN cells).

In [ ]:
def translate(
    text: str,
    tokenizer: Tokenizer,
    model: keras.Model,
    max_len: int = 20
) -> str:
    '''Predicts `text`translation using the `model`.

    Arguments:
        text: text to be translated
        tokenizer: tokenizer to use
        model: model ot use
        max_len: maximum length of the prediction (in tokens)

    Returns:
        tranlated text'''
    ...